# Criando planilhas de campo de forma programática por meio do Pyton

## Contextualização

Neste final de semana inicio mais um trabalho de campo, desta vez de dinâmica florestal. Como de praxe, precisamos criar planilhas para impressão, para que possamos fazer as anotações de coleta de dados.

Em se tratando de dinâmica florestal, a criação de uma planilha de campo a partir de um arquivo do excel pode ser algo dá um certo trabalho,  uma vez que envolve a definção de espaços em branco para inclusão de recrutas, a definição de quebras de páginas conforme mudança de parcelas, a inclusão de novas colunas, etc...

Todo esse trabalho manual poderia ser bem massante, se não fosse a possiblidade de fazer de forma automatizada, usando uma liguagem de programação, como o Python.

A seguir, um passo-a-passo para automatização desta tarefa:

### Pré-processamento

##### 1) Importação dos pacotes necessários

In [2]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.worksheet.pagebreak import Break
from openpyxl.styles import Border, Side

Caso não tenha ainda instalado, usar pip install para instalação das bibliotecas pandas e openpyxl.

##### 2) Importação do arquivo do excel

In [3]:
df = pd.read_excel ('https://github.com/higuchip/planilha_para_impressao/blob/main/arquivo_exemplo.xlsx?raw=true')
df.head()

,T,P,N,ESPÉCIE,Guildas de Regeneração,CAP-2011,CAP-2015,H,x,y,OBS
0,1,1,1,Matayba elaeagnoides Radlk.,CEL,48,50.6,10.0,0.2,3.50,NaN
1,1,1,2,Ocotea pulchella (Nees & Mart.) Mez,CEL,58+22,"59,5+22,5",8.0,0.9,4.40,NaN
2,1,1,3,Matayba elaeagnoides Radlk.,CEL,46.5,46.5,9.5,1.9,3.73,NaN
3,1,1,4,Matayba elaeagnoides Radlk.,CEL,87,87,15.0,0.6,5.16,NaN
4,1,1,5,Matayba elaeagnoides Radlk.,CEL,18,18.2,6.5,0.2,6.70,NaN


##### 3) Remoção dos nomes das autoridades botânicas das espécies

No arquivo utilizado ('dados.xlsx'), para as espécies constam os nomes das autoridades botânicas. No entanto, para uma planilha de campo essa informação é irrelevantente, além de ocupar um amplo espaço horizontal da planilha que será preparada para impressão. Por isso, essa informação pode ser removida.

In [4]:
df['ESPÉCIE']=df['ESPÉCIE'].str.split(' ').str[0] +" " + df['ESPÉCIE'].str.split(' ').str[1]
df.head()

,T,P,N,ESPÉCIE,Guildas de Regeneração,CAP-2011,CAP-2015,H,x,y,OBS
0,1,1,1,Matayba elaeagnoides,CEL,48,50.6,10.0,0.2,3.50,NaN
1,1,1,2,Ocotea pulchella,CEL,58+22,"59,5+22,5",8.0,0.9,4.40,NaN
2,1,1,3,Matayba elaeagnoides,CEL,46.5,46.5,9.5,1.9,3.73,NaN
3,1,1,4,Matayba elaeagnoides,CEL,87,87,15.0,0.6,5.16,NaN
4,1,1,5,Matayba elaeagnoides,CEL,18,18.2,6.5,0.2,6.70,NaN


##### 4) Adicionando a coluna que será preenchida no campo (CAP3)

A coluna CAP3 será preenchida no campo.

In [5]:
df["CAP3"] = " "
df.head()

,T,P,N,ESPÉCIE,Guildas de Regeneração,CAP-2011,CAP-2015,H,x,y,OBS,CAP3
0,1,1,1,Matayba elaeagnoides,CEL,48,50.6,10.0,0.2,3.50,NaN,
1,1,1,2,Ocotea pulchella,CEL,58+22,"59,5+22,5",8.0,0.9,4.40,NaN,
2,1,1,3,Matayba elaeagnoides,CEL,46.5,46.5,9.5,1.9,3.73,NaN,
3,1,1,4,Matayba elaeagnoides,CEL,87,87,15.0,0.6,5.16,NaN,
4,1,1,5,Matayba elaeagnoides,CEL,18,18.2,6.5,0.2,6.70,NaN,


##### 5) Colocando árvores com troncos múltiplos em linhas distintas

No arquivo utilizado, as árvores com tronco múltiplos encontram-se nas mesmas linhas, com os valores de CAP separados pelo sinal de '+'. Dessa maneira, o espaço horizontal para impressão fica muito amplo, de modo que para as planilhas de campo, é mais interessante que os valores de CAP de troncos múltiplos fiquem em linhas distintas.

In [6]:
df['CAP-2015-provisorio'] = df['CAP-2015'].str.split('+')
df = df.explode('CAP-2015-provisorio')
df['CAP-2015-provisorio']=df.apply(lambda row: row['CAP-2015'] if pd.isnull(row['CAP-2015-provisorio']) else row['CAP-2015-provisorio'], axis=1)
df.head()

,T,P,N,ESPÉCIE,Guildas de Regeneração,CAP-2011,CAP-2015,H,x,y,OBS,CAP3,CAP-2015-provisorio
0,1,1,1,Matayba elaeagnoides,CEL,48,50.6,10.0,0.2,3.50,NaN,,50.6
1,1,1,2,Ocotea pulchella,CEL,58+22,"59,5+22,5",8.0,0.9,4.40,NaN,,"59,5"
1,1,1,2,Ocotea pulchella,CEL,58+22,"59,5+22,5",8.0,0.9,4.40,NaN,,"22,5"
2,1,1,3,Matayba elaeagnoides,CEL,46.5,46.5,9.5,1.9,3.73,NaN,,46.5
3,1,1,4,Matayba elaeagnoides,CEL,87,87,15.0,0.6,5.16,NaN,,87


##### 6) Seleção, ordenação e renomeação de colunas

No arquivo original, nem todas as colunas serão úteis para a montagem da planilha de campo. Então neste passo fazemos uma seleção das colunas que serão aproveitadas, assim como renomeamos para melhor referência.

In [7]:
df_subset=df[["T", "P", "N", "ESPÉCIE", 'CAP-2011', "CAP-2015-provisorio", "CAP3", "x", 'y', "OBS"]]
df_subset.rename(columns = {'ESPÉCIE':'Especies', 'CAP-2011':'CAP1', 'CAP-2015-provisorio':'CAP2', 'OBS':'obs'}, inplace = True)
df_subset
df_subset.head()

C:\Users\higuc\AppData\Local\Temp/ipykernel_20820/4003351616.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.rename(columns = {'ESPÉCIE':'Especies', 'CAP-2011':'CAP1', 'CAP-2015-provisorio':'CAP2', 'OBS':'obs'}, inplace = True)


,T,P,N,Especies,CAP1,CAP2,CAP3,x,y,obs
0,1,1,1,Matayba elaeagnoides,48,50.6,,0.2,3.50,NaN
1,1,1,2,Ocotea pulchella,58+22,"59,5",,0.9,4.40,NaN
1,1,1,2,Ocotea pulchella,58+22,"22,5",,0.9,4.40,NaN
2,1,1,3,Matayba elaeagnoides,46.5,46.5,,1.9,3.73,NaN
3,1,1,4,Matayba elaeagnoides,87,87,,0.6,5.16,NaN


##### 7) Remoção de NaN da coluna 'obs'

In [8]:
df_subset['obs'] = df_subset.obs.fillna('')
df_subset

C:\Users\higuc\AppData\Local\Temp/ipykernel_20820/2922497139.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['obs'] = df_subset.obs.fillna('')


,T,P,N,Especies,CAP1,CAP2,CAP3,x,y,obs
0,1,1,1,Matayba elaeagnoides,48,50.6,,0.2,3.50,
1,1,1,2,Ocotea pulchella,58+22,"59,5",,0.9,4.40,
1,1,1,2,Ocotea pulchella,58+22,"22,5",,0.9,4.40,
2,1,1,3,Matayba elaeagnoides,46.5,46.5,,1.9,3.73,
3,1,1,4,Matayba elaeagnoides,87,87,,0.6,5.16,
...,...,...,...,...,...,...,...,...,...,...
173,1,5,C276,Myrcia hatschbachii,NaN,16,,NaN,NaN,Recruta
174,1,5,C269,Cupania vernalis,NaN,16.3,,NaN,NaN,Recruta
175,1,5,C265,Eugenia uruguayensis,NaN,17,,NaN,NaN,Recruta
175,1,5,C265,Eugenia uruguayensis,NaN,16,,NaN,NaN,Recruta


##### 8) Criação do arquivo output_1.xlsx, para posterior geração de arquivo para impressão

In [9]:
df_subset.to_excel('output.xlsx', index = False)


### Geração da planilha para impressão

In [10]:
file = "output.xlsx"
wb = load_workbook(file)
ws = wb.active

# Adicionando recrutas e quebra de pagina

num = 2
while num < ws.max_row:
    if ws["B" + str(num)].value == ws["B" + str(num + 1)].value:
        pass
    elif ws["B" + str(num)].value == ws["J" + str(num)].value:
        pass
    elif ws["B" + str(num)].value != ws["B" + str(num + 1)].value:
        ws.insert_rows(num + 1, 10)
        page_break = Break(id=num+10)  
        ws.row_breaks.append(page_break)  

    num += 1



# inserir bordas

def set_border(ws, cell_range):
    thin = Side(border_style="thin", color="000000")
    for row in ws[cell_range]:
        for cell in row:
            cell.border = Border(top=thin, left=thin, right=thin, bottom=thin)


set_border(ws, "A1:J" + str(ws.max_row))



# Inserir informações e títulos

ws.title = "Planilha de Campo"
ws.print_title_rows = '1:1' # primeira linha


# Salvar
wb.save("planilha_para_impressao.xlsx")


Pronto! A planilha de campo para impressão, 'planilha_para_impressao.xlsx' está pronta. Basta ajustar as larguras das colunas e tamanho de fontes conforme conveniência e mandar imprimir....